In [8]:
# ------------------------------------------------------
# Neo4j 그래프 데이터베이스 생성 시스템
# ------------------------------------------------------

# 필요한 라이브러리 설치 (처음 실행시)
# !pip install neo4j pandas xmltodict

import xml.etree.ElementTree as ET
import pandas as pd
import json
from neo4j import GraphDatabase
from pathlib import Path
import logging
from datetime import datetime
import xmltodict

# 로깅 설정
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# 파일 경로
XML_FILE_PATH = Path("./data/employee_relationships.xml")

# Neo4j 연결 설정
NEO4J_CONFIG = {
    "uri": "bolt://54.162.43.24:7687",            # Neo4j 기본 포트
    "username": "neo4j",                            # 기본 사용자명
    "password": "resident-success-moss"           # 설치시 설정한 비밀번호로 변경 필요
}

class Neo4jManager:
    """Neo4j 데이터베이스 연결 및 관리 클래스 (연결 복구 기능 추가)"""
    
    def __init__(self, uri, username, password):
        self.uri = uri
        self.username = username
        self.password = password
        self.driver = None
        self._connect()
    
    def _connect(self):
        """Neo4j 연결 설정"""
        try:
            # 연결 풀 설정 추가 (안정성 향상)
            self.driver = GraphDatabase.driver(
                self.uri, 
                auth=(self.username, self.password),
                max_connection_lifetime=3600,  # 1시간
                max_connection_pool_size=50,   # 연결 풀 크기
                connection_acquisition_timeout=60  # 연결 획득 타임아웃
            )
            logger.info("Neo4j 연결 성공")
            
            # 연결 테스트
            with self.driver.session() as session:
                result = session.run("RETURN 1")
                logger.info("Neo4j 서버 응답 확인")
                
        except Exception as e:
            logger.error(f"Neo4j 연결 실패: {str(e)}")
            raise
    
    def verify_connection(self):
        """연결 상태 확인 및 복구"""
        try:
            self.driver.verify_connectivity()
            return True
        except Exception as e:
            logger.warning(f"연결 확인 실패: {str(e)}")
            try:
                logger.info("연결 재설정 시도...")
                if self.driver:
                    self.driver.close()
                self._connect()
                return True
            except Exception as reconnect_error:
                logger.error(f"연결 재설정 실패: {str(reconnect_error)}")
                return False
    
    def close(self):
        if self.driver:
            self.driver.close()
            logger.info("Neo4j 연결 종료")
    
    def clear_database(self):
        """데이터베이스 초기화 (기존 데이터 삭제)"""
        with self.driver.session() as session:
            session.run("MATCH (n) DETACH DELETE n")
            logger.info("데이터베이스 초기화 완료")
    
    def create_constraints_and_indexes(self):
        """제약조건 및 인덱스 생성"""
        constraints_queries = [
            "CREATE CONSTRAINT IF NOT EXISTS FOR (e:Employee) REQUIRE e.employee_id IS UNIQUE",
            "CREATE CONSTRAINT IF NOT EXISTS FOR (d:Department) REQUIRE d.name IS UNIQUE",
            "CREATE CONSTRAINT IF NOT EXISTS FOR (p:Project) REQUIRE p.project_id IS UNIQUE",
            "CREATE INDEX IF NOT EXISTS FOR (e:Employee) ON (e.risk_tier)",
            "CREATE INDEX IF NOT EXISTS FOR (e:Employee) ON (e.job_level)",
            "CREATE INDEX IF NOT EXISTS FOR (e:Employee) ON (e.persona)"
        ]
        
        with self.driver.session() as session:
            for query in constraints_queries:
                try:
                    session.run(query)
                    logger.info(f"실행 완료: {query}")
                except Exception as e:
                    logger.warning(f"제약조건/인덱스 생성 실패: {str(e)}")

# Neo4j 매니저 초기화 (실제 환경에 맞게 설정 변경 필요)
print("Neo4j 연결 설정:")
print("1. Neo4j Desktop 또는 Neo4j Community Server가 실행 중이어야 합니다")
print("2. 아래 설정을 실제 환경에 맞게 수정하세요:")
print(f"   - URI: {NEO4J_CONFIG['uri']}")
print(f"   - Username: {NEO4J_CONFIG['username']}")
print(f"   - Password: {NEO4J_CONFIG['password']}")
print("\n주의: 비밀번호를 실제 Neo4j 비밀번호로 변경하세요!")

Neo4j 연결 설정:
1. Neo4j Desktop 또는 Neo4j Community Server가 실행 중이어야 합니다
2. 아래 설정을 실제 환경에 맞게 수정하세요:
   - URI: bolt://54.162.43.24:7687
   - Username: neo4j
   - Password: resident-success-moss

주의: 비밀번호를 실제 Neo4j 비밀번호로 변경하세요!


관계 생성 로직 함수들

In [9]:
# ------------------------------------------------------
# XML 파일 파싱 및 데이터 구조 분석
# ------------------------------------------------------

def parse_xml_to_graph_data(xml_file_path):
    """XML 파일을 파싱하여 그래프 생성용 데이터 추출"""
    
    if not xml_file_path.exists():
        raise FileNotFoundError(f"XML 파일을 찾을 수 없습니다: {xml_file_path}")
    
    try:
        tree = ET.parse(xml_file_path)
        root = tree.getroot()
        
        # 데이터 저장 구조
        graph_data = {
            'employees': [],
            'departments': set(),
            'projects': [],
            'hierarchy_relationships': [],
            'collaboration_relationships': [],
            'project_relationships': []
        }
        
        # 프로젝트 레지스트리 추출
        projects_section = root.find('ProjectRegistry')
        if projects_section is not None:
            for project_elem in projects_section.findall('Project'):
                project_data = {
                    'project_id': project_elem.get('project_id'),
                    'project_name': project_elem.get('project_name', project_elem.get('project_id')),
                    'project_type': project_elem.get('project_type'),
                    'team_size': int(project_elem.get('team_size', 0)),
                    'start_date': project_elem.get('start_date'),
                    'end_date': project_elem.get('end_date'),
                    'status': project_elem.get('status'),
                    'priority': project_elem.get('priority', 'medium')
                }
                graph_data['projects'].append(project_data)
        
        # 직원 정보 및 관계 추출
        employees_section = root.find('Employees')
        if employees_section is not None:
            for emp_elem in employees_section.findall('Employee'):
                
                # 직원 기본 정보
                employee_data = {
                    'employee_id': emp_elem.get('id'),
                    'name': emp_elem.get('name'),
                    'department': emp_elem.get('department'),
                    'job_role': emp_elem.get('job_role'),
                    'job_level': int(emp_elem.get('job_level', 1)),
                    'years_at_company': int(emp_elem.get('years_at_company', 0)),
                    'risk_tier': emp_elem.get('risk_tier'),
                    'persona': emp_elem.get('persona')
                }
                graph_data['employees'].append(employee_data)
                graph_data['departments'].add(emp_elem.get('department'))
                
                # 계층 관계 추출
                hierarchy = emp_elem.find('HierarchyRelationships')
                if hierarchy is not None:
                    # 상사 관계
                    reports_to = hierarchy.find('ReportsTo')
                    if reports_to is not None:
                        hierarchy_rel = {
                            'subordinate_id': emp_elem.get('id'),
                            'manager_id': reports_to.get('manager_id'),
                            'reporting_frequency': reports_to.get('reporting_frequency'),
                            'relationship_start': reports_to.get('reporting_since', '2023-01-01')
                        }
                        graph_data['hierarchy_relationships'].append(hierarchy_rel)
                
                # 협업 관계 추출
                collab = emp_elem.find('CollaborationRelationships')
                if collab is not None:
                    for colleague_elem in collab.findall('Colleague'):
                        collab_rel = {
                            'employee1_id': emp_elem.get('id'),
                            'employee2_id': colleague_elem.get('colleague_id'),
                            'collaboration_strength': float(colleague_elem.get('collaboration_strength', 0)),
                            'interaction_frequency': float(colleague_elem.get('interaction_frequency', 0)),
                            'collaboration_type': colleague_elem.get('collaboration_type'),
                            'relationship_quality': colleague_elem.get('relationship_quality'),
                            'common_projects': int(colleague_elem.get('common_projects', 0))
                        }
                        graph_data['collaboration_relationships'].append(collab_rel)
                
                # 프로젝트 참여 관계 추출
                projects = emp_elem.find('ProjectRelationships')
                if projects is not None:
                    for project_elem in projects.findall('ProjectParticipation'):
                        project_rel = {
                            'employee_id': emp_elem.get('id'),
                            'project_id': project_elem.get('project_id'),
                            'role_in_project': project_elem.get('role_in_project'),
                            'contribution_level': float(project_elem.get('contribution_level', 0)),
                            'project_start': project_elem.get('project_start'),
                            'project_end': project_elem.get('project_end'),
                            'project_status': project_elem.get('project_status')
                        }
                        graph_data['project_relationships'].append(project_rel)
        
        # 부서 정보를 리스트로 변환
        graph_data['departments'] = [{'name': dept} for dept in graph_data['departments']]
        
        logger.info(f"XML 파싱 완료:")
        logger.info(f"  - 직원: {len(graph_data['employees'])}명")
        logger.info(f"  - 부서: {len(graph_data['departments'])}개")
        logger.info(f"  - 프로젝트: {len(graph_data['projects'])}개")
        logger.info(f"  - 계층 관계: {len(graph_data['hierarchy_relationships'])}개")
        logger.info(f"  - 협업 관계: {len(graph_data['collaboration_relationships'])}개")
        logger.info(f"  - 프로젝트 참여: {len(graph_data['project_relationships'])}개")
        
        return graph_data
    
    except Exception as e:
        logger.error(f"XML 파싱 실패: {str(e)}")
        raise

# XML 데이터 파싱 실행
graph_data = parse_xml_to_graph_data(XML_FILE_PATH)

# 샘플 데이터 미리보기
print("\n=== 파싱된 데이터 샘플 ===")
print(f"직원 샘플 (첫 3명):")
for i, emp in enumerate(graph_data['employees'][:3]):
    print(f"  {i+1}. {emp['name']} - {emp['department']} - {emp['job_role']} (Level {emp['job_level']})")

print(f"\n부서 목록:")
for dept in graph_data['departments']:
    print(f"  - {dept['name']}")

print(f"\n프로젝트 샘플 (첫 3개):")
for i, proj in enumerate(graph_data['projects'][:3]):
    print(f"  {i+1}. {proj['project_id']} - {proj['project_type']} - 팀규모: {proj['team_size']}")

INFO:__main__:XML 파싱 완료:
INFO:__main__:  - 직원: 1470명
INFO:__main__:  - 부서: 3개
INFO:__main__:  - 프로젝트: 25개
INFO:__main__:  - 계층 관계: 295개
INFO:__main__:  - 협업 관계: 1098084개
INFO:__main__:  - 프로젝트 참여: 138개



=== 파싱된 데이터 샘플 ===
직원 샘플 (첫 3명):
  1. Employee_1 - Sales - Sales Executive (Level 2)
  2. Employee_2 - Research & Development - Research Scientist (Level 2)
  3. Employee_4 - Research & Development - Laboratory Technician (Level 1)

부서 목록:
  - Research & Development
  - Sales
  - Human Resources

프로젝트 샘플 (첫 3개):
  1. PRJ_001 - single_department - 팀규모: 8
  2. PRJ_002 - single_department - 팀규모: 5
  3. PRJ_003 - cross_department - 팀규모: 8


셀 2: XML 파싱 및 데이터 추출

In [10]:
# ------------------------------------------------------
# Neo4j 노드 생성 함수들
# ------------------------------------------------------

def create_employees_nodes(neo4j_manager, employees_data):
    """직원 노드 생성"""
    
    query = """
    UNWIND $employees as employee
    CREATE (e:Employee {
        employee_id: employee.employee_id,
        name: employee.name,
        department: employee.department,
        job_role: employee.job_role,
        job_level: employee.job_level,
        years_at_company: employee.years_at_company,
        risk_tier: employee.risk_tier,
        persona: employee.persona,
        created_at: datetime()
    })
    """
    
    with neo4j_manager.driver.session() as session:
        result = session.run(query, employees=employees_data)
        logger.info(f"직원 노드 {len(employees_data)}개 생성 완료")

def create_departments_nodes(neo4j_manager, departments_data):
    """부서 노드 생성"""
    
    query = """
    UNWIND $departments as dept
    CREATE (d:Department {
        name: dept.name,
        created_at: datetime()
    })
    """
    
    with neo4j_manager.driver.session() as session:
        result = session.run(query, departments=departments_data)
        logger.info(f"부서 노드 {len(departments_data)}개 생성 완료")

def create_projects_nodes(neo4j_manager, projects_data):
    """프로젝트 노드 생성"""
    
    query = """
    UNWIND $projects as project
    CREATE (p:Project {
        project_id: project.project_id,
        project_name: project.project_name,
        project_type: project.project_type,
        team_size: project.team_size,
        start_date: date(project.start_date),
        end_date: date(project.end_date),
        status: project.status,
        priority: project.priority,
        created_at: datetime()
    })
    """
    
    with neo4j_manager.driver.session() as session:
        result = session.run(query, projects=projects_data)
        logger.info(f"프로젝트 노드 {len(projects_data)}개 생성 완료")

def create_employee_department_relationships(neo4j_manager):
    """직원-부서 관계 생성"""
    
    query = """
    MATCH (e:Employee), (d:Department)
    WHERE e.department = d.name
    CREATE (e)-[:WORKS_IN {
        created_at: datetime()
    }]->(d)
    """
    
    with neo4j_manager.driver.session() as session:
        result = session.run(query)
        summary = result.consume()
        logger.info(f"직원-부서 관계 {summary.counters.relationships_created}개 생성 완료")

print("노드 생성 함수들 정의 완료")

노드 생성 함수들 정의 완료


셀 3: Neo4j 노드 생성 함수들

In [11]:
# ------------------------------------------------------
# Neo4j 노드 생성 함수들
# ------------------------------------------------------

def create_employees_nodes(neo4j_manager, employees_data):
    """직원 노드 생성"""
    
    query = """
    UNWIND $employees as employee
    CREATE (e:Employee {
        employee_id: employee.employee_id,
        name: employee.name,
        department: employee.department,
        job_role: employee.job_role,
        job_level: employee.job_level,
        years_at_company: employee.years_at_company,
        risk_tier: employee.risk_tier,
        persona: employee.persona,
        created_at: datetime()
    })
    """
    
    with neo4j_manager.driver.session() as session:
        result = session.run(query, employees=employees_data)
        logger.info(f"직원 노드 {len(employees_data)}개 생성 완료")

def create_departments_nodes(neo4j_manager, departments_data):
    """부서 노드 생성"""
    
    query = """
    UNWIND $departments as dept
    CREATE (d:Department {
        name: dept.name,
        created_at: datetime()
    })
    """
    
    with neo4j_manager.driver.session() as session:
        result = session.run(query, departments=departments_data)
        logger.info(f"부서 노드 {len(departments_data)}개 생성 완료")

def create_projects_nodes(neo4j_manager, projects_data):
    """프로젝트 노드 생성"""
    
    query = """
    UNWIND $projects as project
    CREATE (p:Project {
        project_id: project.project_id,
        project_name: project.project_name,
        project_type: project.project_type,
        team_size: project.team_size,
        start_date: date(project.start_date),
        end_date: date(project.end_date),
        status: project.status,
        priority: project.priority,
        created_at: datetime()
    })
    """
    
    with neo4j_manager.driver.session() as session:
        result = session.run(query, projects=projects_data)
        logger.info(f"프로젝트 노드 {len(projects_data)}개 생성 완료")

def create_employee_department_relationships(neo4j_manager):
    """직원-부서 관계 생성"""
    
    query = """
    MATCH (e:Employee), (d:Department)
    WHERE e.department = d.name
    CREATE (e)-[:WORKS_IN {
        created_at: datetime()
    }]->(d)
    """
    
    with neo4j_manager.driver.session() as session:
        result = session.run(query)
        summary = result.consume()
        logger.info(f"직원-부서 관계 {summary.counters.relationships_created}개 생성 완료")

print("노드 생성 함수들 정의 완료")

노드 생성 함수들 정의 완료


셀 4: Neo4j 관계 생성 함수들

In [16]:
# ------------------------------------------------------
# Neo4j 관계 생성 함수들
# ------------------------------------------------------

def create_hierarchy_relationships(neo4j_manager, hierarchy_data):
    """계층 관계 생성 (상사-부하)"""
    
    query = """
    UNWIND $relationships as rel
    MATCH (sub:Employee {employee_id: rel.subordinate_id})
    MATCH (mgr:Employee {employee_id: rel.manager_id})
    CREATE (sub)-[:REPORTS_TO {
        reporting_frequency: rel.reporting_frequency,
        relationship_start: CASE 
            WHEN rel.relationship_start IS NOT NULL AND rel.relationship_start <> '' 
            THEN date(rel.relationship_start) 
            ELSE null 
        END,
        created_at: datetime()
    }]->(mgr)
    """
    
    with neo4j_manager.driver.session() as session:
        result = session.run(query, relationships=hierarchy_data)
        summary = result.consume()
        logger.info(f"계층 관계 {summary.counters.relationships_created}개 생성 완료")

def create_collaboration_relationships(neo4j_manager, collaboration_data):
    """협업 관계 생성"""
    
    # 중복 제거: 양방향 관계를 단방향으로 변환
    unique_collaborations = []
    seen_pairs = set()
    
    for collab in collaboration_data:
        emp1, emp2 = collab['employee1_id'], collab['employee2_id']
        pair = tuple(sorted([emp1, emp2]))
        
        if pair not in seen_pairs:
            seen_pairs.add(pair)
            unique_collaborations.append(collab)
    
    # 방향성 관계로 수정하되, 양방향으로 생성하여 무방향 효과 구현
    query = """
    UNWIND $relationships as rel
    MATCH (e1:Employee {employee_id: rel.employee1_id})
    MATCH (e2:Employee {employee_id: rel.employee2_id})
    CREATE (e1)-[:COLLABORATES_WITH {
        collaboration_strength: rel.collaboration_strength,
        interaction_frequency: rel.interaction_frequency,
        collaboration_type: rel.collaboration_type,
        relationship_quality: rel.relationship_quality,
        common_projects: rel.common_projects,
        created_at: datetime()
    }]->(e2)
    CREATE (e2)-[:COLLABORATES_WITH {
        collaboration_strength: rel.collaboration_strength,
        interaction_frequency: rel.interaction_frequency,
        collaboration_type: rel.collaboration_type,
        relationship_quality: rel.relationship_quality,
        common_projects: rel.common_projects,
        created_at: datetime()
    }]->(e1)
    """
    
    with neo4j_manager.driver.session() as session:
        result = session.run(query, relationships=unique_collaborations)
        summary = result.consume()
        logger.info(f"협업 관계 {summary.counters.relationships_created}개 생성 완료")

def create_project_participation_relationships(neo4j_manager, project_relationships_data):
    """프로젝트 참여 관계 생성"""
    
    query = """
    UNWIND $relationships as rel
    MATCH (e:Employee {employee_id: rel.employee_id})
    MATCH (p:Project {project_id: rel.project_id})
    CREATE (e)-[:PARTICIPATES_IN {
        role_in_project: rel.role_in_project,
        contribution_level: rel.contribution_level,
        project_start: CASE 
            WHEN rel.project_start IS NOT NULL AND rel.project_start <> '' 
            THEN date(rel.project_start) 
            ELSE null 
        END,
        project_end: CASE 
            WHEN rel.project_end IS NOT NULL AND rel.project_end <> '' 
            THEN date(rel.project_end) 
            ELSE null 
        END,
        project_status: rel.project_status,
        created_at: datetime()
    }]->(p)
    """
    
    with neo4j_manager.driver.session() as session:
        result = session.run(query, relationships=project_relationships_data)
        summary = result.consume()
        logger.info(f"프로젝트 참여 관계 {summary.counters.relationships_created}개 생성 완료")

def create_project_collaboration_relationships(neo4j_manager):
    """프로젝트 내 팀원 간 협업 관계 생성"""
    
    # 방향성 관계로 수정
    query = """
    MATCH (e1:Employee)-[:PARTICIPATES_IN]->(p:Project)<-[:PARTICIPATES_IN]-(e2:Employee)
    WHERE e1.employee_id < e2.employee_id
    CREATE (e1)-[:PROJECT_COLLEAGUE {
        shared_project: p.project_id,
        created_at: datetime()
    }]->(e2)
    CREATE (e2)-[:PROJECT_COLLEAGUE {
        shared_project: p.project_id,
        created_at: datetime()
    }]->(e1)
    """
    
    with neo4j_manager.driver.session() as session:
        result = session.run(query)
        summary = result.consume()
        logger.info(f"프로젝트 내 협업 관계 {summary.counters.relationships_created}개 생성 완료")

print("관계 생성 함수들 정의 완료")

관계 생성 함수들 정의 완료


셀 5: Neo4j 데이터 베이스 생성 실행

In [17]:
# ------------------------------------------------------
# Neo4j 데이터베이스 생성 실행
# ------------------------------------------------------

def create_neo4j_graph(graph_data, neo4j_config):
    """전체 Neo4j 그래프 생성 프로세스"""
    
    neo4j_manager = None
    
    try:
        # Neo4j 연결
        neo4j_manager = Neo4jManager(
            neo4j_config['uri'], 
            neo4j_config['username'], 
            neo4j_config['password']
        )
        
        # 1. 데이터베이스 초기화
        print("1. 기존 데이터 삭제 중...")
        neo4j_manager.clear_database()
        
        # 2. 제약조건 및 인덱스 생성
        print("2. 제약조건 및 인덱스 생성 중...")
        neo4j_manager.create_constraints_and_indexes()
        
        # 3. 노드 생성
        print("3. 노드 생성 중...")
        create_departments_nodes(neo4j_manager, graph_data['departments'])
        create_employees_nodes(neo4j_manager, graph_data['employees'])
        create_projects_nodes(neo4j_manager, graph_data['projects'])
        
        # 4. 기본 관계 생성 (직원-부서)
        print("4. 기본 관계 생성 중...")
        create_employee_department_relationships(neo4j_manager)
        
        # 5. 조직 관계 생성
        print("5. 조직 관계 생성 중...")
        
        # 계층 관계 생성
        if graph_data.get('hierarchy_relationships'):
            print(f"   5-1. 계층 관계 {len(graph_data['hierarchy_relationships'])}개 생성 중...")
            create_hierarchy_relationships(neo4j_manager, graph_data['hierarchy_relationships'])
        
        # 협업 관계 생성 (대용량 데이터 배치 처리)
        if graph_data.get('collaboration_relationships'):
            collab_data = graph_data['collaboration_relationships']
            print(f"   5-2. 협업 관계 {len(collab_data)}개 생성 중... (배치 처리)")
            
            # 대용량 데이터를 배치로 나누어 처리
            batch_size = 10000  # 10K씩 배치 처리
            total_batches = (len(collab_data) + batch_size - 1) // batch_size
            
            for i in range(0, len(collab_data), batch_size):
                batch = collab_data[i:i+batch_size]
                batch_num = (i // batch_size) + 1
                print(f"      배치 {batch_num}/{total_batches} 처리 중... ({len(batch)}개)")
                
                try:
                    create_collaboration_relationships(neo4j_manager, batch)
                except Exception as e:
                    logger.warning(f"배치 {batch_num} 처리 중 오류 발생: {str(e)}")
                    # 배치 실패 시 계속 진행
                    continue
        
        # 프로젝트 참여 관계 생성
        if graph_data.get('project_relationships'):
            print(f"   5-3. 프로젝트 참여 관계 {len(graph_data['project_relationships'])}개 생성 중...")
            create_project_participation_relationships(neo4j_manager, graph_data['project_relationships'])
        
        # 6. 파생 관계 생성 (프로젝트 내 협업)
        print("6. 파생 관계 생성 중...")
        try:
            create_project_collaboration_relationships(neo4j_manager)
        except Exception as e:
            logger.warning(f"프로젝트 내 협업 관계 생성 실패: {str(e)}")
            print("   프로젝트 내 협업 관계 생성을 건너뜁니다.")
        
        # 7. 생성된 그래프 통계
        print("7. 그래프 통계 확인 중...")
        get_graph_statistics(neo4j_manager)
        
        print("\n✅ Neo4j 그래프 생성 완료!")
        
        return neo4j_manager
        
    except Exception as e:
        logger.error(f"Neo4j 그래프 생성 실패: {str(e)}")
        if neo4j_manager:
            neo4j_manager.close()
        raise

def get_graph_statistics(neo4j_manager):
    """생성된 그래프의 통계 정보 조회"""
    
    statistics_queries = {
        "총 노드 수": "MATCH (n) RETURN count(n) as count",
        "총 관계 수": "MATCH ()-[r]->() RETURN count(r) as count",
        "직원 노드 수": "MATCH (e:Employee) RETURN count(e) as count",
        "부서 노드 수": "MATCH (d:Department) RETURN count(d) as count", 
        "프로젝트 노드 수": "MATCH (p:Project) RETURN count(p) as count",
        "계층 관계 수": "MATCH ()-[r:REPORTS_TO]->() RETURN count(r) as count",
        "협업 관계 수 (방향별)": "MATCH ()-[r:COLLABORATES_WITH]->() RETURN count(r) as count",
        "협업 관계 수 (고유 쌍)": """
            MATCH (e1:Employee)-[r:COLLABORATES_WITH]->(e2:Employee)
            WHERE e1.employee_id < e2.employee_id
            RETURN count(r) as count
        """,
        "프로젝트 참여 관계 수": "MATCH ()-[r:PARTICIPATES_IN]->() RETURN count(r) as count",
        "프로젝트 내 협업 관계 수": "MATCH ()-[r:PROJECT_COLLEAGUE]->() RETURN count(r) as count",
        "직원-부서 관계 수": "MATCH ()-[r:WORKS_IN]->() RETURN count(r) as count"
    }
    
    print("\n=== Neo4j 그래프 통계 ===")
    
    with neo4j_manager.driver.session() as session:
        for description, query in statistics_queries.items():
            try:
                result = session.run(query)
                count = result.single()["count"]
                print(f"  {description}: {count:,}")
            except Exception as e:
                print(f"  {description}: 조회 실패 - {str(e)}")

def validate_graph_integrity(neo4j_manager):
    """그래프 데이터 무결성 검증"""
    
    validation_queries = {
        "고아 직원 (부서 없음)": """
            MATCH (e:Employee)
            WHERE NOT (e)-[:WORKS_IN]->(:Department)
            RETURN count(e) as count
        """,
        "매니저 역할 불일치": """
            MATCH (e:Employee)<-[:REPORTS_TO]-()
            WHERE NOT exists(e.employee_id)
            RETURN count(e) as count
        """,
        "중복 협업 관계": """
            MATCH (e1:Employee)-[r1:COLLABORATES_WITH]->(e2:Employee),
                  (e1)-[r2:COLLABORATES_WITH]->(e2)
            WHERE id(r1) <> id(r2)
            RETURN count(r1) as count
        """
    }
    
    print("\n=== 데이터 무결성 검증 ===")
    
    with neo4j_manager.driver.session() as session:
        for description, query in validation_queries.items():
            try:
                result = session.run(query)
                count = result.single()["count"]
                status = "⚠️" if count > 0 else "✅"
                print(f"  {status} {description}: {count:,}")
            except Exception as e:
                print(f"  ❌ {description}: 검증 실패 - {str(e)}")

# Neo4j 그래프 생성 실행
# 주의: 실제 Neo4j 서버가 실행 중이고, 올바른 인증 정보가 설정되어 있어야 합니다.

try:
    print("Neo4j 그래프 생성 시작...")
    print(f"Neo4j 서버: {NEO4J_CONFIG['uri']}")
    print(f"처리 대상 데이터:")
    print(f"  - 직원: {len(graph_data['employees']):,}명")
    print(f"  - 부서: {len(graph_data['departments']):,}개")  
    print(f"  - 프로젝트: {len(graph_data['projects']):,}개")
    print(f"  - 계층관계: {len(graph_data['hierarchy_relationships']):,}개")
    print(f"  - 협업관계: {len(graph_data['collaboration_relationships']):,}개")
    print(f"  - 프로젝트참여: {len(graph_data['project_relationships']):,}개")
    print("-" * 60)
    
    # 실제 실행
    neo4j_manager = create_neo4j_graph(graph_data, NEO4J_CONFIG)
    
    # 추가 검증
    if neo4j_manager:
        validate_graph_integrity(neo4j_manager)
        
        print("\n🎯 그래프 생성 성공!")
        print("Neo4j Browser에서 시각화를 확인하세요:")
        print("http://localhost:7474")
    
except Exception as e:
    print(f"\n❌ Neo4j 연결 실패: {str(e)}")
    print("\n해결 방법:")
    print("1. Neo4j Desktop 또는 Community Server가 실행 중인지 확인")
    print("2. 연결 정보(URI, username, password)가 올바른지 확인")
    print("3. 방화벽에서 7687 포트가 열려있는지 확인")
    print("4. 메모리 부족 시 협업 관계 배치 크기 줄이기 (batch_size = 5000)")
    print("\n설정을 확인한 후 다시 실행하세요.")
    neo4j_manager = None

INFO:__main__:Neo4j 연결 성공


Neo4j 그래프 생성 시작...
Neo4j 서버: bolt://54.162.43.24:7687
처리 대상 데이터:
  - 직원: 1,470명
  - 부서: 3개
  - 프로젝트: 25개
  - 계층관계: 295개
  - 협업관계: 1,098,084개
  - 프로젝트참여: 138개
------------------------------------------------------------


INFO:__main__:Neo4j 서버 응답 확인


1. 기존 데이터 삭제 중...


INFO:__main__:데이터베이스 초기화 완료


...

   5-3. 프로젝트 참여 관계 138개 생성 중...


INFO:__main__:프로젝트 참여 관계 138개 생성 완료


6. 파생 관계 생성 중...


INFO:__main__:프로젝트 내 협업 관계 708개 생성 완료


7. 그래프 통계 확인 중...

=== Neo4j 그래프 통계 ===
  총 노드 수: 1,498
  총 관계 수: 2,189,547
  직원 노드 수: 1,470
  부서 노드 수: 3
  프로젝트 노드 수: 25
  계층 관계 수: 295
  협업 관계 수 (방향별): 2,186,936
  협업 관계 수 (고유 쌍): 1,093,468
  프로젝트 참여 관계 수: 138
  프로젝트 내 협업 관계 수: 708
  직원-부서 관계 수: 1,470

✅ Neo4j 그래프 생성 완료!

=== 데이터 무결성 검증 ===
  ✅ 고아 직원 (부서 없음): 0
  ❌ 매니저 역할 불일치: 검증 실패 - {code: Neo.ClientError.Statement.SyntaxError} {message: The property existence syntax `... exists(variable.property)` is no longer supported. Please use `variable.property IS NOT NULL` instead. (line 3, column 23 (offset: 72))
"            WHERE NOT exists(e.employee_id)"
                       ^}
  ⚠️ 중복 협업 관계: 2,177,704

🎯 그래프 생성 성공!
Neo4j Browser에서 시각화를 확인하세요:
http://localhost:7474


셀 6: Neo4j 쿼리 예시들

In [18]:
# ------------------------------------------------------
# Neo4j 쿼리 예시들 (그래프 활용)
# ------------------------------------------------------

def run_sample_queries(neo4j_manager):
    """Neo4j 그래프 쿼리 예시 실행"""
    
    if not neo4j_manager:
        print("Neo4j 연결이 없습니다. 먼저 그래프를 생성하세요.")
        return
    
    sample_queries = {
        "고위험 직원 조회": """
            MATCH (e:Employee)
            WHERE e.risk_tier = 'High'
            RETURN e.employee_id, e.name, e.department, e.job_role, e.persona
            ORDER BY e.employee_id
            LIMIT 10
        """,
        
        "관리자별 부하직원 수": """
            MATCH (manager:Employee)<-[:REPORTS_TO]-(subordinate:Employee)
            RETURN manager.employee_id as manager_id, 
                   manager.name as manager_name,
                   manager.department as department,
                   count(subordinate) as subordinate_count
            ORDER BY subordinate_count DESC
            LIMIT 10
        """,
        
        "부서별 협업 네트워크 밀도": """
            MATCH (e1:Employee)-[:COLLABORATES_WITH]-(e2:Employee)
            WHERE e1.department = e2.department
            WITH e1.department as dept, count(*) as collab_count
            MATCH (emp:Employee)
            WHERE emp.department = dept
            WITH dept, collab_count, count(emp) as emp_count
            RETURN dept, emp_count, collab_count,
                   round(toFloat(collab_count) / (emp_count * (emp_count - 1) / 2), 3) as density
            ORDER BY density DESC
        """,
        
        "가장 활발한 협업자": """
            MATCH (e:Employee)-[r:COLLABORATES_WITH]-()
            RETURN e.employee_id, e.name, e.department, 
                   count(r) as collaboration_count,
                   round(avg(r.collaboration_strength), 3) as avg_strength
            ORDER BY collaboration_count DESC
            LIMIT 10
        """,
        
        "프로젝트별 팀 구성": """
            MATCH (p:Project)<-[rel:PARTICIPATES_IN]-(e:Employee)
            WHERE p.status = 'active'
            RETURN p.project_id, p.project_name, p.project_type,
                   collect({
                       employee_id: e.employee_id, 
                       name: e.name, 
                       role: rel.role_in_project,
                       contribution: rel.contribution_level
                   }) as team_members
            ORDER BY p.project_id
            LIMIT 5
        """,
        
        "위험 직원의 관계 네트워크": """
            MATCH (high_risk:Employee {risk_tier: 'High'})
            OPTIONAL MATCH (high_risk)-[r1:REPORTS_TO]->(manager)
            OPTIONAL MATCH (high_risk)-[r2:COLLABORATES_WITH]-(colleague)
            OPTIONAL MATCH (high_risk)-[r3:PARTICIPATES_IN]->(project)
            RETURN high_risk.employee_id as employee_id,
                   high_risk.name as name,
                   high_risk.persona as persona,
                   manager.name as manager_name,
                   collect(DISTINCT colleague.name)[0..3] as top_colleagues,
                   collect(DISTINCT project.project_id)[0..3] as active_projects
            LIMIT 10
        """
    }
    
    print("=== Neo4j 쿼리 예시 실행 ===\n")
    
    with neo4j_manager.driver.session() as session:
        for query_name, query in sample_queries.items():
            try:
                print(f"📊 {query_name}:")
                result = session.run(query)
                records = list(result)
                
                if records:
                    # 결과를 DataFrame으로 변환하여 보기 좋게 출력
                    df_result = pd.DataFrame([record.data() for record in records])
                    print(df_result.to_string(index=False, max_colwidth=50))
                else:
                    print("   결과 없음")
                    
                print("\n" + "-"*80 + "\n")
                
            except Exception as e:
                print(f"   쿼리 실행 실패: {str(e)}\n")

def create_cypher_queries_for_cognita():
    """Cognita 에이전트용 특화 쿼리들"""
    
    cognita_queries = {
        "사회적 고립 직원 탐지": """
            MATCH (e:Employee)
            OPTIONAL MATCH (e)-[c:COLLABORATES_WITH]-()
            WITH e, count(c) as collab_count
            WHERE collab_count <= 2
            RETURN e.employee_id, e.name, e.department, e.risk_tier, collab_count
            ORDER BY collab_count, e.risk_tier DESC
        """,
        
        "관리자 변경이 필요한 팀 탐지": """
            MATCH (manager:Employee)<-[:REPORTS_TO]-(sub:Employee)
            WHERE sub.risk_tier = 'High'
            WITH manager, count(sub) as high_risk_count, count(*) as total_subs
            WHERE high_risk_count >= 2
            RETURN manager.employee_id, manager.name, manager.department,
                   total_subs, high_risk_count,
                   round(toFloat(high_risk_count)/total_subs, 2) as risk_ratio
            ORDER BY risk_ratio DESC
        """,
        
        "프로젝트 위험도 분석": """
            MATCH (p:Project)<-[:PARTICIPATES_IN]-(e:Employee)
            WITH p, 
                 sum(CASE WHEN e.risk_tier = 'High' THEN 1 ELSE 0 END) as high_risk_count,
                 count(e) as total_members
            WHERE total_members > 0
            RETURN p.project_id, p.project_name, p.status,
                   total_members, high_risk_count,
                   round(toFloat(high_risk_count)/total_members, 2) as project_risk_ratio
            ORDER BY project_risk_ratio DESC
        """
    }
    
    return cognita_queries

# 쿼리 예시 실행 (Neo4j 연결이 있을 때만)
if 'neo4j_manager' in locals() and neo4j_manager:
    run_sample_queries(neo4j_manager)
    
    print("\n=== Cognita 에이전트 특화 쿼리 ===")
    cognita_queries = create_cypher_queries_for_cognita()
    
    with neo4j_manager.driver.session() as session:
        for query_name, query in cognita_queries.items():
            try:
                print(f"\n🔍 {query_name}:")
                result = session.run(query)
                records = list(result)
                
                if records:
                    df_result = pd.DataFrame([record.data() for record in records])
                    print(df_result.head().to_string(index=False))
                else:
                    print("   결과 없음")
                    
            except Exception as e:
                print(f"   쿼리 실행 실패: {str(e)}")
else:
    print("Neo4j 연결이 없어 쿼리 예시를 건너뜁니다.")
    print("연결 설정 후 이 셀을 다시 실행하세요.")

=== Neo4j 쿼리 예시 실행 ===

📊 고위험 직원 조회:
e.employee_id        e.name           e.department             e.job_role                   e.persona
            1    Employee_1                  Sales        Sales Executive Onboarding Failure (온보딩 실패)
         1001 Employee_1001 Research & Development  Laboratory Technician Onboarding Failure (온보딩 실패)
         1002 Employee_1002 Research & Development  Laboratory Technician          Burnout (저성과자/번아웃)
         1003 Employee_1003                  Sales        Sales Executive          Burnout (저성과자/번아웃)
         1004 Employee_1004 Research & Development     Research Scientist          Burnout (저성과자/번아웃)
         1005 Employee_1005 Research & Development Manufacturing Director          Burnout (저성과자/번아웃)
         1006 Employee_1006 Research & Development     Research Scientist Onboarding Failure (온보딩 실패)
         1007 Employee_1007 Research & Development Manufacturing Director Onboarding Failure (온보딩 실패)
          101  Employee_101 Research & Develo

셀 7: Neo4j Browser 연동 및 시각화 가이드

In [19]:
# ------------------------------------------------------
# Neo4j Browser 시각화 가이드
# ------------------------------------------------------

def generate_visualization_queries():
    """Neo4j Browser에서 사용할 시각화 쿼리들"""
    
    viz_queries = {
        "전체 조직 구조 (샘플)": """
            // 한 부서의 계층 구조 시각화
            MATCH (e:Employee)-[:WORKS_IN]->(d:Department {name: 'Sales'})
            OPTIONAL MATCH (e)-[r:REPORTS_TO]->(manager)
            RETURN e, r, manager, d
            LIMIT 20
        """,
        
        "고위험 직원과 그 관계": """
            // 고위험 직원과 연결된 모든 관계
            MATCH (high_risk:Employee {risk_tier: 'High'})
            OPTIONAL MATCH (high_risk)-[r1]-(connected)
            RETURN high_risk, r1, connected
            LIMIT 50
        """,
        
        "프로젝트 협업 네트워크": """
            // 특정 프로젝트의 팀원과 그들의 협업 관계
            MATCH (p:Project {status: 'active'})<-[:PARTICIPATES_IN]-(e:Employee)
            OPTIONAL MATCH (e)-[c:COLLABORATES_WITH]-(colleague)-[:PARTICIPATES_IN]->(p)
            RETURN p, e, c, colleague
            LIMIT 30
        """,
        
        "부서간 협업 네트워크": """
            // 서로 다른 부서 직원들 간의 협업 관계
            MATCH (e1:Employee)-[:WORKS_IN]->(d1:Department)
            MATCH (e2:Employee)-[:WORKS_IN]->(d2:Department)
            MATCH (e1)-[c:COLLABORATES_WITH]-(e2)
            WHERE d1 <> d2 AND c.collaboration_strength > 0.6
            RETURN e1, e2, c, d1, d2
            LIMIT 25
        """,
        
        "위험 전파 경로 분석": """
            // 고위험 직원으로부터 2단계 이내의 관계
            MATCH (high_risk:Employee {risk_tier: 'High'})
            MATCH path = (high_risk)-[*1..2]-(connected:Employee)
            WHERE connected.risk_tier IN ['High', 'Neutral']
            RETURN path
            LIMIT 40
        """
    }
    
    print("=== Neo4j Browser 시각화 가이드 ===\n")
    
    print("1. Neo4j Browser 접속:")
    print("   http://localhost:7474 (기본 주소)")
    print("   username: neo4j")
    print("   password: [설정한 비밀번호]\n")
    
    print("2. 아래 쿼리들을 복사해서 Neo4j Browser에서 실행하세요:\n")
    
    for name, query in viz_queries.items():
        print(f"🎨 {name}:")
        print("```cypher")
        print(query.strip())
        print("```\n")
    
    print("3. 시각화 팁:")
    print("   - 노드 크기: 직급(job_level) 또는 위험도(risk_tier)에 따라 조정")
    print("   - 노드 색상: 부서별 또는 위험등급별로 구분")
    print("   - 관계 두께: 협업 강도(collaboration_strength)에 따라 조정")
    print("   - 레이아웃: Force-directed 또는 Hierarchical 레이아웃 사용")
    
    return viz_queries

# 시각화 쿼리 생성
viz_queries = generate_visualization_queries()

# Neo4j 연결 정리
if 'neo4j_manager' in locals() and neo4j_manager:
    print(f"\n=== Neo4j 그래프 생성 완료 ===")
    print(f"✅ 총 {len(graph_data['employees'])}명의 직원 데이터가 Neo4j에 저장되었습니다.")
    print(f"✅ 계층관계, 협업관계, 프로젝트 참여 관계가 모두 생성되었습니다.")
    print(f"\n🌐 Neo4j Browser에서 그래프를 시각적으로 탐색해보세요:")
    print(f"   http://localhost:7474")
    
    # 필요한 경우 연결 종료
    # neo4j_manager.close()
else:
    print("\n⚠️  Neo4j 연결 설정을 완료한 후 다시 실행하세요.")
    print("   1. Neo4j 서버 실행")
    print("   2. 인증 정보 수정")
    print("   3. 코드 재실행")

=== Neo4j Browser 시각화 가이드 ===

1. Neo4j Browser 접속:
   http://localhost:7474 (기본 주소)
   username: neo4j
   password: [설정한 비밀번호]

2. 아래 쿼리들을 복사해서 Neo4j Browser에서 실행하세요:

🎨 전체 조직 구조 (샘플):
```cypher
// 한 부서의 계층 구조 시각화
            MATCH (e:Employee)-[:WORKS_IN]->(d:Department {name: 'Sales'})
            OPTIONAL MATCH (e)-[r:REPORTS_TO]->(manager)
            RETURN e, r, manager, d
            LIMIT 20
```

🎨 고위험 직원과 그 관계:
```cypher
// 고위험 직원과 연결된 모든 관계
            MATCH (high_risk:Employee {risk_tier: 'High'})
            OPTIONAL MATCH (high_risk)-[r1]-(connected)
            RETURN high_risk, r1, connected
            LIMIT 50
```

🎨 프로젝트 협업 네트워크:
```cypher
// 특정 프로젝트의 팀원과 그들의 협업 관계
            MATCH (p:Project {status: 'active'})<-[:PARTICIPATES_IN]-(e:Employee)
            OPTIONAL MATCH (e)-[c:COLLABORATES_WITH]-(colleague)-[:PARTICIPATES_IN]->(p)
            RETURN p, e, c, colleague
            LIMIT 30
```

🎨 부서간 협업 네트워크:
```cypher
// 서로 다른 부서 직원들 간의 협업 관계
            MATCH (e1: